#**ATP Match Data Clean and Database Creation**

Using what we learned from exploring the data, we'll now create an analysis database that removes some of the troubling observations, repopulates some missing data, and some other pre-processing. First, we mount my google drive where I've saved the data we'll be using. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We then import the packages we'll need. 


In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter 
import os
import glob
import seaborn as sns
import datetime
import re
os.chdir("/content/drive/My Drive/ATP Tennis/Data/")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Then we read in the csv data into pandas data frames. We'll use these to clean the data before creating our analysis database. The data are ATP tour-level main draw singles matches, players, and rankings. Most of the information in the player and ranking tables is already be in the match data.

**Player Data**

We read in the player data. We don't do any processing of this data.

In [3]:
atp_players = pd.read_csv("atp_players.csv",names=["player","first_name","last_name","hand","birth_date","country_code"])

**Ranking Data**

We add year, month, and day fields to the ranking data. We also standardize some column names. Finally, we drop duplicates. Looking at the duplicates, we see that they are exact duplicates and so no information is lost by "deduping". 

In [4]:
filenames = [i for i in glob.glob('*.{}'.format('csv')) if "atp_rankings" in i]
atp_rankings = pd.concat([pd.read_csv(f) for f in filenames if f !="atp_rankings_current.csv"])
atp_rankings_current = pd.read_csv("atp_rankings_current.csv",names=atp_rankings.columns)
atp_rankings = pd.concat([atp_rankings,atp_rankings_current])
atp_rankings["year"] = [int(np.floor(i/10000)) for i in atp_rankings["ranking_date"].values]
atp_rankings['monthday'] = atp_rankings['ranking_date'] - atp_rankings['year']*10000
atp_rankings['month'] = [int(np.floor(i/100)) for i in atp_rankings['monthday']]
atp_rankings['day'] = atp_rankings['monthday'] - atp_rankings['month']*100
del atp_rankings['monthday']
atp_rankings = atp_rankings.drop_duplicates()

**Match Data**

After reading in the match data, we add a year field to the to keep track of the file source as well as the year.

In [5]:
filenames = [i for i in glob.glob('*.{}'.format('csv')) if "atp_matches" in i]
atp_matches = pd.concat([pd.read_csv(f) for f in filenames ])
atp_matches["year"] = [int(i[0:4]) for i in atp_matches["tourney_id"].values]
atp_matches.shape

(176951, 50)

We identified a series of ways in which we might want to clean up the match data in the data exploration notebook. These include the following bullets. See that notebook for more detail on these. 

- Drop matches with no detail match statistics
- Drop Davis Cup matches
- Drop matches with missing minutes
- Drop matches with missing ranking points and rankings
- Create set scores from `score` field

Let's actually work through some of that cleaning now. 

First, we create a flag for matches with missing detail match statistics. This can be done by just filtering based on one of the detail match statistics. We'll use winner ace count (`w_ace`). We also create a flag for all Davis Cup matches useing the `tourney_level` field. Finally, we create a flag for matches with missing `minutes`  (most of these are from 2015 and before 2000). We also create a flag for matches with missing player ranks or points. These are primarily from before 2000. 

In [6]:
atp_matches["NoDetailStats"] = np.where(atp_matches["w_ace"].notnull(),0,1)
atp_matches["Davis_Cup"] = np.where(atp_matches["tourney_level"] != "D",0,1)
atp_matches["NoMinutes"] = np.where(atp_matches["minutes"].notnull(),0,1)
atp_matches["NoWinnerRank"] = np.where(atp_matches["winner_rank"].notnull(),0,1)
atp_matches["NoLoserRank"] = np.where(atp_matches["loser_rank"].notnull(),0,1)
atp_matches["NoWinnerRankPoints"] = np.where(atp_matches["winner_rank_points"].notnull(),0,1)
atp_matches["NoLoserRankPoints"] = np.where(atp_matches["loser_rank_points"].notnull(),0,1)

Here is code that can be used for actually dropping these matches. We're still left with over 80,000 matches. These are all from after 1991 however, when detail match stats are available. 

In [7]:
atp_matches[(atp_matches["NoDetailStats"]==0) & 
            (atp_matches["Davis_Cup"]==0) & 
            (atp_matches["NoMinutes"]==0) & 
            (atp_matches["NoWinnerRank"]==0) & 
            (atp_matches["NoLoserRank"]==0) & 
            (atp_matches["NoWinnerRankPoints"]==0) & 
            (atp_matches["NoLoserRankPoints"]==0)].shape

(81309, 57)

Now, let's create set by set scores from the `score` field. We'll also identify when a player retired, drop matches with suspicious or obviously wrong scores, and add a field indicating whether there was a tiebreak and what the score was. 

First we create a retirement indicator. We see that ~2% of matches ended in retirement.

In [8]:
atp_matches["retirement"] = [1 if str(i).find("RET") != -1 else 0 for i in atp_matches["score"]]
x = Counter(atp_matches["retirement"])
print("Number of Matches Ending in Retirement:",x[1])
print("Percent of Matches Ending in Retirement:",round(x[1]/x[0]*100,1),"%")

Number of Matches Ending in Retirement: 3477
Percent of Matches Ending in Retirement: 2.0 %


Next we flag any matches with a score that includes a letter other than "RET". We drop these. There are relatively few. We actually drop them so that we can easily create the set level score fields.

In [9]:
score = [str(s).replace('RET', '').strip() for s in atp_matches["score"]]
any_alpha = [re.search('[a-zA-Z]', str(s)) for s in score]
drop_any_alpha = [s is None for s in any_alpha]
print(Counter(drop_any_alpha))
atp_matches = atp_matches[drop_any_alpha]
atp_matches = atp_matches[atp_matches["score"]!=""]
atp_matches.shape

Counter({True: 175536, False: 1415})


(175536, 58)

Note that there are a few matches with retirements before one game was finished.

In [10]:
score = [str(s).replace('RET', '').strip() for s in atp_matches["score"]]
atp_matches[[True if s=="" else False for s in score ]]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,NoDetailStats,Davis_Cup,NoMinutes,NoWinnerRank,NoLoserRank,NoWinnerRankPoints,NoLoserRankPoints,retirement
353,1982-402,Memphis,Carpet,64,A,19820208,286,100376,NaN,NaN,Buster C Mottram,R,NaN,GBR,26.792608,100456,NaN,Q,Ferdi Taygan,R,173.0,USA,25.177276,RET,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,NaN,NaN,NaN,1982,1,0,1,0,1,1,1,1
2012,2007-520,Roland Garros,Clay,128,G,20070528,8,104022,13,NaN,Mikhail Youzhny,R,183.0,RUS,24.922656,103401,NaN,NaN,Jan Hernych,R,190.0,CZE,27.890486,RET,5,R128,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,5.0,3.0,1.0,0.0,1.0,1.0,15.0,1685.0,92.0,454.0,2007,0,0,0,0,0,0,0,1
125,2013-338,Sydney,Hard,28,A,20130107,14,103898,NaN,NaN,Julien Benneteau,R,185.0,FRA,31.049966,103285,6,NaN,Radek Stepanek,R,185.0,CZE,34.113621,RET,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,1075.0,34.0,1090.0,2013,1,0,1,0,0,0,0,1
2663,1971-313,Bristol,Grass,32,A,19710607,280,100073,NaN,NaN,Mark Cox,L,NaN,GBR,27.923340,100100,NaN,NaN,Tony Roche,L,178.0,AUS,26.056126,RET,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1971,1,0,1,1,1,1,1,1


Now I wtire some code to break the overall score into set by set scores. Note that this creates many missing values for third, fourth, and fifth sets, which are not always played. We then break the set level scores into the number of games won in that set for each player. Note that the winner of the match's games are listed first. Also note that the results of tie breaks are in parentheses. The tie break score is just the number of points won by the loser of the tie break. We move these into another seperate column. So we end up with three columns per set: winner games, loser games, and tie break score.

Note that I don't actually run this code yet, since it adds many fields but hasn't been used for any analyses. This may change.

In [ ]:
set_scores = pd.DataFrame.from_records([str(s).split(" ") for s in score],
                                       columns=["Set1","Set2","Set3","Set4","Set5"])
def getSetScores(n):
  Set_score = pd.DataFrame.from_records([s.replace(')', '').split("(") for s in 
              [str(s) for s in set_scores["Set"+str(n)].values]],
              columns=["Set"+str(n)+"_Score","Set"+str(n)+"_Tiebreak"])
  Set_Tiebreak = Set_score["Set"+str(n)+"_Tiebreak"]
  Set_score = pd.DataFrame.from_records([s.split("-") for s in Set_score["Set"+str(n)+"_Score"]],
              columns=["Winner_Set"+str(n)+"_Score","Loser_Set"+str(n)+"_Score"])
  Set_score["Set"+str(n)+"_Tiebreak"] = Set_Tiebreak
  atp_matches["Winner_Set"+str(n)+"_Score"] = Set_score["Winner_Set"+str(n)+"_Score"]
  atp_matches["Loser_Set"+str(n)+"_Score"] = Set_score["Loser_Set"+str(n)+"_Score"]
  atp_matches["Set"+str(n)+"_Tiebreak"] = Set_score["Set"+str(n)+"_Tiebreak"]

Set1 = getSetScores(1)
Set2 = getSetScores(2)
Set3 = getSetScores(3)
Set4 = getSetScores(4)
Set5 = getSetScores(5)

Now I export a pickle with data. I also have some code to create a SQL database but the pickle is pretty easy considering most of the data we want is already in the match data and there arent too many seperate tables.  

In [11]:
import pickle
atp_pickle = [atp_matches,atp_players,atp_rankings]
with open('/content/drive/My Drive/ATP Tennis/Database/ATP_Pickle.pickle', 'wb') as f:
    pickle.dump(atp_pickle, f)

**Create SQL Database**

Here is code to create a `sqlite3` database that will store the match, player, and ranking information. 

I dont use this for now. 


In [12]:
connection = sqlite3.connect('/content/drive/My Drive/ATP Tennis/Database/ATP_Tennis.db')
c = connection.cursor()

And then we create tables in the database. 

In [13]:
atp_matches.to_sql('ATP_MATCHES', connection, if_exists='replace', index = False)
atp_players.to_sql('ATP_PLAYERS', connection, if_exists='replace', index = False)
atp_rankings.to_sql('ATP_RANKINGS', connection, if_exists='replace', index = False)

So we now have a cleaned analysis database that we can use for analysis. We'll get into that in future notebooks.